In [26]:
%matplotlib inline

import pandas_datareader.data as web
import datetime as datetime
import numpy as np
import pandas as pd
from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.signal import lfilter

orig_df = web.DataReader('JPNRGDPEXP', 'fred' ,start = datetime.datetime(1994,1,1),end = datetime.datetime(2021,1,1))
df = orig_df.copy()
df

,JPNRGDPEXP
DATE,
1994-01-01,446278.8
1994-04-01,443805.7
1994-07-01,448938.7
1994-10-01,447131.8
1995-01-01,452083.4
...,...
2020-01-01,544231.2
2020-04-01,500232.3
2020-07-01,526697.2


In [27]:
df = df.dropna(how='any')
index = pd.date_range(df.index[0], periods=df.shape[0], freq="QS")
df.set_index(index, inplace=True)

In [28]:
# model parameters
nobs = len(df)
phi_vec = np.r_[0.5, 0.5] #phi1, phi2
sigma_sq = np.r_[10, 10, 10] #sigma_tau, sigma_c, sigma_beta

In [29]:
"""
Univariate Local Linear Trend Model
"""
class LocalLinearTrend(sm.tsa.statespace.MLEModel):
    def __init__(self, endog):
        # Model order
        k_states = 4
        k_posdef = 4

        # Initialize the statespace
        super(LocalLinearTrend, self).__init__(
            endog, k_states=k_states, k_posdef=k_posdef,
            initialization='approximate_diffuse',
            loglikelihood_burn=k_states
        )

        # Initialize the matrices
        self.ssm['design'] = np.array([1, 1, 0, 0])
        self.ssm['transition'] = np.array([[1, 0, 1, 0],
                                       [0, phi_vec[0], 0, phi_vec[1]],
                                       [0, 0, 1, 0],
                                       [0, 0, 0, 1]])
        self.ssm['selection'] = np.eye(k_states)
        self.ssm['selection', 3, 3] = 0

        # Cache some indices
        self._state_cov_idx = ('state_cov',) + np.diag_indices(k_posdef)

    @property
    def param_names(self):
        return ['phi.1', 'phi.2', 'sigma2.tau', 'sigma2.c', 'sigma2.beta']

    @property
    def start_params(self):
        return [phi_vec[0], phi_vec[1], sigma_sq[0], sigma_sq[1], sigma_sq[2]]

    def transform_params(self, unconstrained):
        # only variance 
        unconstrained[2:] = unconstrained[2:]**2
        return unconstrained

    def untransform_params(self, constrained):
        # only variance
        constrained[2:] = constrained[2:]**0.5
        return constrained

    def update(self, params, *args, **kwargs):
        params = super(LocalLinearTrend, self).update(params, *args, **kwargs)
        self['transition', 1, 1] = params[0]
        self['transition', 1, 3] = params[1]
        self['state_cov', 0, 0] = params[2]
        self['state_cov', 1, 1] = params[3]
        self['state_cov', 2, 2] = params[4]

In [31]:
# Create and fit the model
mod = LocalLinearTrend(df)
res = mod.fit(disp=False)
res.summary()

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:             JPNRGDPEXP   No. Observations:                  109
Model:               LocalLinearTrend   Log Likelihood               -1093.942
Date:                Sun, 11 Jul 2021   AIC                           2197.885
Time:                        18:26:16   BIC                           2211.155
Sample:                    01-01-1994   HQIC                          2203.262
                         - 01-01-2021                                         
Covariance Type:                  opg                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
phi.1           0.3020      0.061      4.918      0.000       0.182       0.422
phi.2          42.0563    772.575      0.054      0.957   -1472.162    1556.275
sigma2.tau   3.799e+06   5.99e+05      6.337      0.000    2.62e+06    4.97e+06
sigma2.c      2.02e+07   7.47e+05     27.025      0.000    1.87e+07    2.17e+07
sigma2.beta  1.117e+07   1.38e+06      8.091      0.000    8.46e+06    1.39e+07
===================================================================================
Ljung-Box (L1) (Q):                   0.66   Jarque-Bera (JB):               534.87
Prob(Q):                              0.42   Prob(JB):                         0.00
Heteroskedasticity (H):               9.86   Skew:                             0.27
Prob(H) (two-sided):                  0.00   Kurtosis:                        14.04
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number 1.38e+17. Standard errors may be unstable.
"""

In [25]:
# Perform prediction and forecasting
predict = res.get_prediction(start='1994Q1', end='2021Q1', dynamic=True)
predict_df = predict.summary_frame(alpha=0.05)
predict_df

JPNRGDPEXP,mean,mean_se,mean_ci_lower,mean_ci_upper
1994-01-01,0.0,1.414214e+03,-2.771808e+03,2.771808e+03
1994-04-01,0.0,1.712105e+04,-3.355664e+04,3.355664e+04
1994-07-01,0.0,2.217958e+04,-4.347118e+04,4.347118e+04
1994-10-01,0.0,2.463145e+04,-4.827676e+04,4.827676e+04
1995-01-01,0.0,2.716603e+04,-5.324445e+04,5.324445e+04
...,...,...,...,...
2019-01-01,0.0,1.949693e+06,-3.821328e+06,3.821328e+06
2019-04-01,0.0,1.979129e+06,-3.879021e+06,3.879021e+06
2019-07-01,0.0,2.008711e+06,-3.937001e+06,3.937001e+06
2019-10-01,0.0,2.038439e+06,-3.995267e+06,3.995267e+06
